In [10]:
# Web Scraping - LinkedIn

import streamlit as st
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from time import sleep, time
import lxml
import re
from urllib.parse import urljoin
import unicodedata

from datetime import datetime 


# cit = input('Please, enter a city:\n')
# stat = input('Please, enter a state:\n')
# city = str(cit.replace(' ', '+'))
# state = str(stat.replace(' ','+'))
# location = city+"%2C%20"+state

# print(f'Searching {city},{state}. Please wait...')

# search_url = "https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location="+location+"&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0&f_TP=1%2C2"

data = {'title': [],
        'company': [], 
        'location': [],
        'description': [],
        'date': [],}


def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    clean = re.sub(cleanr, ' ', str(raw_html))
    cleaner = clean.strip()
    cleantext = re.sub('\n', ' ', cleaner)
    return cleantext


def desc_clean(text):
    clean = unicodedata.normalize("NFKD", str(text))
    testy = clean.replace('\\xa0','')
    return testy


def export_table(data):
    table = pd.DataFrame(data, columns=['title', 'company', 'location', 'description', 'date'])
    table.index = table.index + 1
    # table.to_csv('/users/dmauger/Flatiron/FinalProject/' + 'total_data.csv', mode='a', encoding='utf-8', index=False)
    # gpl.load(source='dataframe', destination='bq', data_name='total_data', dataframe=table, write_disposition='WRITE_APPEND', compress=False)
    return table 
    
    # print('Scraping done. Here are the results:')
    # print(table.info())

def export_storage(table):
    table.to_csv('/users/dmauger/Flatiron/FinalProject/' + 'total_data_date.csv', mode='a', encoding='utf-8', index=False)
    # gpl.load(source='local', destination='bq', data_name='total_data', write_disposition='WRITE_APPEND',bq_schema=bq_schema)
    print('Process completed.')

def job_details(job):
    
    sleep(1)

    r = requests.get(job)
    r.encoding = 'utf-8'

    html_content = r.text
    soup = BeautifulSoup(html_content, 'html.parser')
    info = soup.find('div', class_='topcard__content')

    try:
        title = info.find('h1').text
    except:
        title = 'NaN'

    try:
        company = soup.find('div', class_='topcard__content').find_all('span')[0].text
    except:
        company = 'NaN'

    try:
        location = info.find('h3', class_='topcard__flavor-row').find_all('span')[1].text
    except:
        location = 'NaN'

    try:
        description = soup.find('div', class_="description__text")
    except:
        description = soup.find('div', class_="description__text").find_all('p')

    data['title'].append(cleanhtml(title))
    data['company'].append(cleanhtml(company))
    data['location'].append(cleanhtml(location))
    data['description'].append(desc_clean(cleanhtml(description)))
    data['date'] = pd.to_datetime('today')


def extract_title(li_url):

    sleep(1)
    page = requests.get(li_url)
    page.encoding = 'utf-8'
    bs = BeautifulSoup(page.content, 'html.parser')

    links = []
    for div in bs.find('div', class_="results__container").find_all('li'):
        for a in div.find_all('a', attrs={'href': re.compile("^https://www.linkedin.com/jobs/")}):
            links.append(a['href'])

    for job in links:
        job_details(job)

    return export_table(data)



# extract_title(search_url)


# cit = input('Please, enter a city:\n')
# stat = input('Please, enter a state:\n')
# city = str(cit.replace(' ', '+'))
# state = str(stat.replace(' ','+'))
# location = city+"%2C%20"+state

# print(f'Searching {city},{state}. Please wait...')

# search_url = "https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location="+location+"&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0&f_TP=1%2C2"

In [11]:
url = "https://www.linkedin.com/jobs/search?keywords=Data%20Engineer&location=Atlanta%2C%20Georgia%2C%20United%20States&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"

In [12]:
test2 = extract_title(url)

In [14]:
test2.head(100)

,title,company,location,description,date
1,Data Engineer,Cox Automotive Inc.,"Atlanta, GA, US",We are looking for a Data Engineer to join the...,2019-12-15 14:13:10.377599
2,Data Engineer,TechTalent Squared,"Norcross, Georgia",TechTalent Squared is seeking a Data Engineer ...,2019-12-15 14:13:10.377599
3,Data Engineer,Auth0,"Atlanta, GA, US",Auth0 is a pre-IPO unicorn. We are growing rap...,2019-12-15 14:13:10.377599
4,Data Engineer,Applied Resource Group,"Atlanta, Georgia","Location: Atlanta, GA Do you want to...",2019-12-15 14:13:10.377599
5,Data Engineer,eHire,"Atlanta, GA, US",Data Engineer Apply Today! Do you enjoy help...,2019-12-15 14:13:10.377599
6,Data Engineer,Luxoft,"Alpharetta, GA, US",Description Project Description About the P...,2019-12-15 14:13:10.377599
7,Data Engineer,"New Relic, Inc.","Atlanta, GA, US",Your Opportunity We are looking for a Data ...,2019-12-15 14:13:10.377599
8,Data Engineer,"CapTech Ventures, Inc","Atlanta, GA, US",Company Description CapTech is an award-win...,2019-12-15 14:13:10.377599
9,Data Engineer,Kforce Inc,"Atlanta, GA, US",Responsibilities Kforce has a client seekin...,2019-12-15 14:13:10.377599
10,Data Engineer,Jobot,"Atlanta, GA, US",This Jobot Job is hosted by Greg Smith Are you...,2019-12-15 14:13:10.377599


In [20]:
test2.to_csv('/users/dmauger/Flatiron/FinalProject/' + 'total_data_date.csv', mode='a', encoding='utf-8', index=False)

In [21]:
total2 = pd.read_csv('total_data_date.csv')


In [25]:
total2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8958 entries, 0 to 8957
Data columns (total 5 columns):
title          8889 non-null object
company        8452 non-null object
location       8452 non-null object
description    8958 non-null object
date           8958 non-null object
dtypes: object(5)
memory usage: 350.0+ KB


In [6]:
total['date'] = pd.to_datetime('today')

In [26]:
total2.to_csv('/users/dmauger/Flatiron/FinalProject/' + 'total_data.csv', mode='a', encoding='utf-8', index=False)

In [27]:
test = pd.read_csv('total_data.csv')
test.head()

ParserError: Error tokenizing data. C error: Expected 4 fields in line 8931, saw 5
